In [1]:
# All Necessary Imports
import torch 
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_confusion_matrix,classification_report, accuracy_score , confusion_matrix

In [2]:
# Importing Dataset into colab 
from google.colab import files
data = files.upload()

Saving iris.csv to iris.csv


In [3]:
!ls

iris.csv  sample_data


In [4]:
# Importing Dataset into Dataframe 
df = pd.read_csv('iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [6]:
# Here You can see that we have 3 classes 
# Here -> we can either use label encoding in target or one hot encoding.
# Label Encoding will create one target column 
# One hot encoding will create three target columns 

# I will show both approches along with sequential and functional implementation. 
df.species.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

# Data Preparation 

In [7]:
# Now we all know that torch uses float32 tensor and numpy has float64 
# Also dimension should be (Rows, cols) so I wll reshape Data 
# With small dataset you will ANN generally overfits but here we will ignore this As we are learning 
len(df)

150

In [8]:
X = df.drop('species',axis=1)
y = df['species']

In [9]:
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [10]:
y.head()

0    setosa
1    setosa
2    setosa
3    setosa
4    setosa
Name: species, dtype: object

In [11]:
# Label Encoding : -
# Pytorch supports label encoding but it should start from 0 to n 
# Here we have 3 species so we will create target map 

target_map = {'setosa':0,'versicolor':1,'virginica':2}
target_map

{'setosa': 0, 'versicolor': 1, 'virginica': 2}

In [12]:
y.apply(lambda x: target_map[x])

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: species, Length: 150, dtype: int64

In [13]:
y = y.apply(lambda x: target_map[x])

In [14]:
# Now let's Reshape and convert data
# Dont run this cell twice , It will give Error 
X = X.values.reshape(X.shape[0],X.shape[1]).astype(np.float32)
y = y.values.reshape(y.shape[0],-1).astype(np.float32)

print('Shape :', X.shape,' Dtype:', X.dtype)
print('Shape :', y.shape,' Dtype:', y.dtype)

Shape : (150, 4)  Dtype: float32
Shape : (150, 1)  Dtype: float32


In [15]:
X[0:5,:]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]], dtype=float32)

In [16]:
y[0:5]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [17]:
# Convert numpy to tensor 
print(type(X))
print(type(y))

print('--Conversion---')
X = torch.from_numpy(X)
y = torch.from_numpy(y)
print(type(X))
print(type(y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
--Conversion---
<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [18]:
# Now lets split Data into Train and Test 
X_train, X_test , y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=101)
len(X_train), len(X_test) , len(y_train), len(y_test)

(112, 38, 112, 38)

In [19]:
# iterable Creation 
train_data = TensorDataset(X_train,y_train)
test_data = TensorDataset(X_test,y_test)

# loader Creation 
train_data_loader = DataLoader(train_data,batch_size=32,shuffle=True)
test_data_loader = DataLoader(test_data,batch_size=38)

# Let's Create and Train Model 
### Type 1 : Linear classification simple linear model

In [132]:
 # Linear Model -- > Contains Only 1 Input and 1 output layer 
 # This model can only generate st lines 
 class Linear_NN(nn.Module):
   def __init__(self,input_shape,output_shape):
     super(Linear_NN,self).__init__()
     self.layers = nn.Linear(input_shape,output_shape)
   def forward(self,x):
     return self.layers(x)

In [133]:
model = Linear_NN(4,3)
model

Linear_NN(
  (layers): Linear(in_features=4, out_features=3, bias=True)
)

In [134]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [135]:
n_epoch = 500
train_acc = []
train_loss = []

for epoch in range(n_epoch):
  if epoch%100 == 0:
    print(f'Epoch : {epoch+1}/{n_epoch} ')

  for batch , data in enumerate(train_data_loader):
    x = data[0]
    y = data[1]
    
    optimizer.zero_grad()   # Removing accumulation of gradients 
    y_pred = model(x)       # model forward pass
    loss = criterion(y_pred,y.squeeze().long())  # loss calculation 
    train_loss.append(loss.item())  
    loss.backward()  # df/dx
    optimizer.step() # w = w - n*df/dx

    
    with torch.no_grad():
      accuracy = (torch.argmax(y_pred,axis=1) == y.squeeze().long()).float().mean()
    train_acc.append(accuracy)

    if epoch%100 == 0:
      print(f'    Batch : {batch}  Loss : {loss.item():.4f}  Accuracy :{accuracy}')

Epoch : 1/500 
    Batch : 0  Loss : 2.9493  Accuracy :0.34375
    Batch : 1  Loss : 3.0822  Accuracy :0.375
    Batch : 2  Loss : 2.6525  Accuracy :0.34375
    Batch : 3  Loss : 2.3498  Accuracy :0.3125
Epoch : 101/500 
    Batch : 0  Loss : 0.8225  Accuracy :0.65625
    Batch : 1  Loss : 0.8271  Accuracy :0.59375
    Batch : 2  Loss : 0.8417  Accuracy :0.625
    Batch : 3  Loss : 0.9044  Accuracy :0.6875
Epoch : 201/500 
    Batch : 0  Loss : 0.6096  Accuracy :0.6875
    Batch : 1  Loss : 0.5533  Accuracy :0.78125
    Batch : 2  Loss : 0.6245  Accuracy :0.6875
    Batch : 3  Loss : 0.5756  Accuracy :0.875
Epoch : 301/500 
    Batch : 0  Loss : 0.4667  Accuracy :0.84375
    Batch : 1  Loss : 0.5291  Accuracy :0.78125
    Batch : 2  Loss : 0.4731  Accuracy :0.84375
    Batch : 3  Loss : 0.4905  Accuracy :0.6875
Epoch : 401/500 
    Batch : 0  Loss : 0.4656  Accuracy :0.78125
    Batch : 1  Loss : 0.4174  Accuracy :0.90625
    Batch : 2  Loss : 0.4274  Accuracy :0.90625
    Batch : 3  L

In [154]:
with torch.no_grad():
  x,y = test_data[0:]
  y_pred = model(x)
  loss = criterion(y_pred,y.squeeze().long())

  accuracy = (torch.argmax(y_pred,axis=1) == y.squeeze().long()).float().mean()

  print(f'Test Loss : {loss.item():.4f}  Accuracy : {accuracy} ')


Test Loss : 0.4684  Accuracy : 0.9736841917037964 


In [143]:
# Test 
test_ = torch.tensor([5.1,3.5,1.4,0.2]).reshape(1,4)
test_

tensor([[5.1000, 3.5000, 1.4000, 0.2000]])

In [153]:
result = model(test_)
print('Actual Target : ',target_map.items(),'\nResult : ',torch.argmax(result,axis=1).item())

Actual Target :  dict_items([('setosa', 0), ('versicolor', 1), ('virginica', 2)]) 
Result :  0


In [159]:
print(classification_report(torch.argmax(y_pred,axis=1),y.squeeze().long()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.94      1.00      0.97        16
           2       1.00      0.92      0.96        12

    accuracy                           0.97        38
   macro avg       0.98      0.97      0.98        38
weighted avg       0.98      0.97      0.97        38



### Type 2 : Linear classification Non Linear Model (Sequential)

In [176]:
class Linear_NN(nn.Module):
   def __init__(self,input_shape,output_shape):
     super(Linear_NN,self).__init__()
     self.layers = nn.Sequential(nn.Linear(input_shape,16),
                                 nn.Sigmoid(),
                                 nn.Linear(16,output_shape))
   def forward(self,x):
     return self.layers(x)

In [177]:
model = Linear_NN(4,3)
model

Linear_NN(
  (layers): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=16, out_features=3, bias=True)
  )
)

In [178]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [179]:
n_epoch = 500
train_acc = []
train_loss = []

for epoch in range(n_epoch):
  if epoch%100 == 0:
    print(f'Epoch : {epoch+1}/{n_epoch} ')

  for batch , data in enumerate(train_data_loader):
    x = data[0]
    y = data[1]
    
    optimizer.zero_grad()   # Removing accumulation of gradients 
    y_pred = model(x)       # model forward pass
    loss = criterion(y_pred,y.squeeze().long())  # loss calculation 
    train_loss.append(loss.item())  
    loss.backward()  # df/dx
    optimizer.step() # w = w - n*df/dx

    
    with torch.no_grad():
      accuracy = (torch.argmax(y_pred,axis=1) == y.squeeze().long()).float().mean()
    train_acc.append(accuracy)

    if epoch%100 == 0:
      print(f'    Batch : {batch}  Loss : {loss.item():.4f}  Accuracy :{accuracy}')


Epoch : 1/500 
    Batch : 0  Loss : 1.1963  Accuracy :0.34375
    Batch : 1  Loss : 1.2619  Accuracy :0.28125
    Batch : 2  Loss : 1.1226  Accuracy :0.40625
    Batch : 3  Loss : 1.0877  Accuracy :0.375
Epoch : 101/500 
    Batch : 0  Loss : 0.5986  Accuracy :0.59375
    Batch : 1  Loss : 0.5115  Accuracy :0.78125
    Batch : 2  Loss : 0.5312  Accuracy :0.8125
    Batch : 3  Loss : 0.5233  Accuracy :0.75
Epoch : 201/500 
    Batch : 0  Loss : 0.3085  Accuracy :0.96875
    Batch : 1  Loss : 0.3706  Accuracy :0.9375
    Batch : 2  Loss : 0.4130  Accuracy :0.96875
    Batch : 3  Loss : 0.4313  Accuracy :1.0
Epoch : 301/500 
    Batch : 0  Loss : 0.2671  Accuracy :0.9375
    Batch : 1  Loss : 0.2806  Accuracy :0.96875
    Batch : 2  Loss : 0.2101  Accuracy :1.0
    Batch : 3  Loss : 0.3350  Accuracy :1.0
Epoch : 401/500 
    Batch : 0  Loss : 0.2346  Accuracy :0.9375
    Batch : 1  Loss : 0.1670  Accuracy :1.0
    Batch : 2  Loss : 0.1419  Accuracy :1.0
    Batch : 3  Loss : 0.2060  Accu

In [180]:
with torch.no_grad():
  x,y = test_data[0:]
  y_pred = model(x)
  loss = criterion(y_pred,y.squeeze().long())

  accuracy = (torch.argmax(y_pred,axis=1) == y.squeeze().long()).float().mean()

  print(f'Test Loss : {loss.item():.4f}  Accuracy : {accuracy} ')


Test Loss : 0.1492  Accuracy : 1.0 


In [181]:
print(classification_report(torch.argmax(y_pred,axis=1),y.squeeze().long()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        17
           2       1.00      1.00      1.00        11

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



In [182]:
print(confusion_matrix(torch.argmax(y_pred,axis=1),y.squeeze().long()))

[[10  0  0]
 [ 0 17  0]
 [ 0  0 11]]


### Type 3 : Linear classification Non Linear Model (Functional)

In [188]:
class Linear_NN(nn.Module):
   def __init__(self,input_shape,output_shape):
     super(Linear_NN,self).__init__()
     self.layer1 = nn.Linear(input_shape,16)
     self.activation = nn.Sigmoid()
     self.layer2 = nn.Linear(16,output_shape)

   def forward(self,x):
     x = self.layer1(x)
     x = self.activation(x)
     x = self.layer2(x)
     return x

In [189]:
model = Linear_NN(4,3)
model

Linear_NN(
  (layer1): Linear(in_features=4, out_features=16, bias=True)
  (activation): Sigmoid()
  (layer2): Linear(in_features=16, out_features=3, bias=True)
)

In [190]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [191]:
n_epoch = 500
train_acc = []
train_loss = []

for epoch in range(n_epoch):
  if epoch%100 == 0:
    print(f'Epoch : {epoch+1}/{n_epoch} ')

  for batch , data in enumerate(train_data_loader):
    x = data[0]
    y = data[1]
    
    optimizer.zero_grad()   # Removing accumulation of gradients 
    y_pred = model(x)       # model forward pass
    loss = criterion(y_pred,y.squeeze().long())  # loss calculation 
    train_loss.append(loss.item())  
    loss.backward()  # df/dx
    optimizer.step() # w = w - n*df/dx

    
    with torch.no_grad():
      accuracy = (torch.argmax(y_pred,axis=1) == y.squeeze().long()).float().mean()
    train_acc.append(accuracy)

    if epoch%100 == 0:
      print(f'    Batch : {batch}  Loss : {loss.item():.4f}  Accuracy :{accuracy}')

Epoch : 1/500 
    Batch : 0  Loss : 1.0837  Accuracy :0.40625
    Batch : 1  Loss : 1.1605  Accuracy :0.28125
    Batch : 2  Loss : 1.1160  Accuracy :0.40625
    Batch : 3  Loss : 1.1117  Accuracy :0.3125
Epoch : 101/500 
    Batch : 0  Loss : 0.5017  Accuracy :0.78125
    Batch : 1  Loss : 0.5480  Accuracy :0.75
    Batch : 2  Loss : 0.4966  Accuracy :0.84375
    Batch : 3  Loss : 0.5699  Accuracy :0.75
Epoch : 201/500 
    Batch : 0  Loss : 0.3771  Accuracy :0.90625
    Batch : 1  Loss : 0.3672  Accuracy :1.0
    Batch : 2  Loss : 0.3474  Accuracy :1.0
    Batch : 3  Loss : 0.2594  Accuracy :0.875
Epoch : 301/500 
    Batch : 0  Loss : 0.2469  Accuracy :1.0
    Batch : 1  Loss : 0.2765  Accuracy :0.90625
    Batch : 2  Loss : 0.1888  Accuracy :1.0
    Batch : 3  Loss : 0.2195  Accuracy :1.0
Epoch : 401/500 
    Batch : 0  Loss : 0.1301  Accuracy :1.0
    Batch : 1  Loss : 0.1456  Accuracy :0.96875
    Batch : 2  Loss : 0.1685  Accuracy :0.96875
    Batch : 3  Loss : 0.2721  Accuracy

In [192]:
with torch.no_grad():
  x,y = test_data[0:]
  y_pred = model(x)
  loss = criterion(y_pred,y.squeeze().long())

  accuracy = (torch.argmax(y_pred,axis=1) == y.squeeze().long()).float().mean()

  print(f'Test Loss : {loss.item():.4f}  Accuracy : {accuracy} ')

Test Loss : 0.1301  Accuracy : 1.0 


In [193]:
print(classification_report(torch.argmax(y_pred,axis=1),y.squeeze().long()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        17
           2       1.00      1.00      1.00        11

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



In [194]:
print(confusion_matrix(torch.argmax(y_pred,axis=1),y.squeeze().long()))

[[10  0  0]
 [ 0 17  0]
 [ 0  0 11]]


In [195]:
# Conclusion 
# 1 . Linear model took 500 epochs to get good accuracy but could reach up to 80-90%
# 2. Non linear took same epoch but able to get accuracy of 100% 

# Data has some non linearity in it which we are able to find with the help of Non linear model